# Blocking with Word2Vec

In [8]:
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Caricamento dello Schema Mediato da File

In [34]:
schema_mediato = pd.read_csv("./../../datasets/MediatedSchemaSemicolon.csv",sep=";")
print("Dimensioni dello Schema Mediato:", schema_mediato.shape)

Dimensioni dello Schema Mediato: (81706, 6)


In [35]:
records = []
for i in range(0,len(schema_mediato)):
    record = schema_mediato.iloc[i].dropna()
    records.append(record.tolist())

In [36]:
records

[['Lacework', 'United States, San Jose', '$8.3B', 'Cybersecurity', '2015'],
 ['Tipalti', 'United States, San Mateo', '$8.3B', 'Fintech', '2010'],
 ['Tempus', 'United States, Chicago', '$8.1B', 'Health', '2015'],
 ['Anduril',
  'United States, Irvine',
  '$8.48B',
  'Artificial intelligence',
  '2017'],
 ['Bolt', 'Estonia, Tallinn', '$8.4B', 'Auto & transportation', '2013'],
 ['ByteDance', 'China, Beijing', '$140B', 'Artificial intelligence', '2012'],
 ['Niantic',
  'United States, San Francisco',
  '$9B',
  'Mobile & telecommunications',
  '2015'],
 ['OYO Rooms', 'India, Gurugram', '$9B', 'Travel', '2012'],
 ['Kavak',
  'Mexico, Lerma de Villada',
  '$8.7B',
  'E-commerce & direct-to-consumer',
  '2014'],
 ['Personio',
  'Germany, Munich',
  '$8.5B',
  'Internet software & services',
  '2015'],
 ['Klaviyo',
  'United States, Boston',
  '$9.2B',
  'Internet software & services',
  '2012'],
 ['Rapyd', 'United Kingdom, London', '$8.75B', 'Fintech', '2016'],
 ['Nuro',
  'United States, Mou

Osservazione: ogni Record è rappresentato con una Lista di Stringhe, dove le 'stringhe' sono i campi che lo caratterizzano.

## Inizializzazione del Modello "Word2Ved"

In [11]:
num_threads = os.cpu_count()
print("Numero di thread disponibili:", num_threads)

Numero di thread disponibili: 8


In [60]:
model = Word2Vec(sentences=records, vector_size=1024, window=5, min_count=1, workers=num_threads)

## Calcolo dei Vettori per i Record
Abbiamo settato la dimensione dello Spazio Latente a 256 e per ogni vettore dobbiamo, quindi, ottenere un vettore di 256 valori.

Per un Record il vettore viene ottenuto sommando tra loro "element-wise" i vettori ottenuti dai campi ed infine normalizzando il vettore risultante sul valore 256, pari alla dimensione dello Spazio Latente.

In [6]:
#STRATEGIA 1: somma normalizzata dei token vettorizzati
def calculate_vector(record):
    # Inizializza un vettore vuoto
    record_vector = np.zeros(model.vector_size)
    # Somma i vettori di embedding di ciascun token nel record
    for token in record:
        record_vector += model.wv[token]
    #Normalizza il vettore risultante
    record_vector /= 512
    return record_vector

In [61]:
schema_mediato[['name', 'location', 'marketCap', 'sharePrice', 'category', 'foundationYear']] = schema_mediato[['name', 'location', 'marketCap', 'sharePrice', 'category', 'foundationYear']].astype(str)
schema_mediato['combined'] = schema_mediato[['name', 'location', 'marketCap', 'sharePrice', 'category', 'foundationYear']].apply(lambda x: ' '.join(x), axis=1)
# Tokenizzazione
tokenized_data = schema_mediato['combined'].apply(lambda x: x.lower().split())

model = Word2Vec(sentences=tokenized_data, vector_size=512, window=5, min_count=1, workers=4)

In [62]:
# Funzione per generare embedding per i record
def record_embedding(record_tokens, model):
    embeddings = []
    for token in record_tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return None

In [63]:
schema_mediato['embedding'] = tokenized_data.apply(lambda x: record_embedding(x, model))

print(schema_mediato[['combined', 'embedding']])

                                                combined  \
0      Lacework United States, San Jose $8.3B nan Cyb...   
1      Tipalti United States, San Mateo $8.3B nan Fin...   
2      Tempus United States, Chicago $8.1B nan Health...   
3      Anduril United States, Irvine $8.48B nan Artif...   
4      Bolt Estonia, Tallinn $8.4B nan Auto & transpo...   
...                                                  ...   
81701  PepsiCo Purchase, NY, United States $241.05 Bi...   
81702  Costco Issaquah, WA, United States $258.15 Bil...   
81703  Broadcom San Jose, CA, United States $265.27 B...   
81704  Coca-Cola Atlanta, GA, United States $273.20 B...   
81705  Lilly Indianapolis, IN, United States $273.01 ...   

                                               embedding  
0      [0.39524814, 0.06270445, -0.014102078, 0.18386...  
1      [0.39582717, 0.09236683, 0.003069281, 0.210677...  
2      [0.37436157, 0.017520823, -0.06787337, 0.14383...  
3      [0.30459693, 0.048026636, -0.0374700

In [64]:
schema_mediato['embedding']

0        [0.39524814, 0.06270445, -0.014102078, 0.18386...
1        [0.39582717, 0.09236683, 0.003069281, 0.210677...
2        [0.37436157, 0.017520823, -0.06787337, 0.14383...
3        [0.30459693, 0.048026636, -0.03747003, 0.15145...
4        [0.1504307, -0.040354397, 0.05225756, 0.342248...
                               ...                        
81701    [0.33947957, 0.06857674, 0.07836795, 0.2916877...
81702    [0.37351885, 0.040589247, 0.102169275, 0.32061...
81703    [0.41305602, 0.11504476, 0.08066246, 0.2736404...
81704    [0.37550136, 0.07651665, 0.07325151, 0.3152286...
81705    [0.34228042, 0.06907375, 0.07845143, 0.299124,...
Name: embedding, Length: 81706, dtype: object

In [33]:
records_w2v = []
for i in range(0, len(records)):
    records_w2v.append(calculate_vector(records[i]))

Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia
Aia


In [8]:
df = pd.DataFrame(records_w2v)
print("Dimensioni del DataFrame di Vettori:", df.shape)

Dimensioni del DataFrame di Vettori: (81706, 512)


## K-Means sui Vettori per costruire i Blocchi

In [65]:
embeddings = schema_mediato['embedding'].to_numpy()
embeddings

array([array([ 3.95248145e-01,  6.27044514e-02, -1.41020780e-02,  1.83867931e-01,
              -5.62470295e-02,  5.59018105e-02, -2.88585853e-02,  9.18577984e-02,
              -8.44965354e-02, -3.22847009e-01, -5.92570901e-02, -1.95961609e-01,
              -2.87210375e-01, -1.31432906e-01,  3.34026664e-01,  1.14338145e-01,
              -5.60651958e-01, -1.05157718e-01,  5.70653439e-01,  1.19749002e-01,
               1.54280737e-01, -1.26873523e-01, -1.34545699e-01,  7.96291828e-02,
              -4.62664872e-01, -1.28740631e-02, -1.66225806e-01, -1.39394268e-01,
              -1.37516066e-01,  3.43723744e-02, -5.97656518e-02, -3.17875713e-01,
              -3.96298647e-01, -3.56302798e-01, -5.32997139e-02,  6.77488670e-02,
               1.00285895e-01,  1.12439364e-01, -2.60163546e-01, -1.59404539e-02,
               1.53109089e-01, -2.57943720e-01,  2.40711510e-01,  1.62034556e-01,
              -1.62542045e-01, -2.14664459e-01,  6.21460825e-02, -1.95407093e-01,
              -1

In [66]:
embeddings = np.array([np.array(embedding) for embedding in schema_mediato['embedding']])

In [67]:
kmeans = KMeans(n_clusters=2000, random_state=0).fit(embeddings)

/Users/alessandropesare/anaconda3/envs/FinalProjectIDD3_8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [68]:
schema_mediato['block'] = kmeans.labels_

In [69]:
schema_mediato.to_csv("./../../datasets/MediatedSchemaConBlocks_W2V.csv", index=False)